In [1]:
import pandas as pd
import numpy as np
import h5py
import math
import itertools
from itertools import product, combinations

from tokenizers import ByteLevelBPETokenizer

In [2]:
samples = pd.read_csv('Reviews.csv')

In [3]:
samples['x'] = samples['Review'].str.lower()
samples['x'] = samples['x'].str.replace(r'[^a-z\s\']', ' ', regex=True)
samples['x'] = samples['x'].str.replace(r'\'', '', regex=True)
samples['x'] = samples['x'].str.replace(r'\s+', ' ', regex=True)
samples['x']

0        working with one of the best shakespeare sourc...
1        well tremors i the original started off in and...
2        ouch this one was a bit painful to sit through...
3        ive seen some crappy movies in my life but thi...
4         carriers follows the exploits of two guys and...
                               ...                        
49995    this movie is certainly well constructed begin...
49996    nice to see a comedy for grown ups masterfully...
49997    jean renoirs homage to the paris of the late t...
49998    what are the movies i mean what are movies mad...
49999    i saw this movie on tv and loved it i am a rea...
Name: x, Length: 50000, dtype: object

In [4]:
tokenizer = ByteLevelBPETokenizer()
tokenizer.train_from_iterator(samples['x'], vocab_size=1024, min_frequency=256)

In [5]:
vocabulary = tokenizer.get_vocab()
vocubulary_size = len(vocabulary)
vocubulary_size

1024

In [6]:
samples['x'] = samples['x'].apply(lambda x: tokenizer.encode(x).ids)
sequence_size = samples['x'].apply(len).max()
samples['x'] = samples['x'].apply(lambda x: (([0]*sequence_size)+x)[-sequence_size:])

sequence_size

4913

In [7]:
samples['x']

0        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                               ...                        
49995    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
49996    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
49997    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
49998    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
49999    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: x, Length: 50000, dtype: object

In [8]:
import pandas as pd
import numpy as np
from itertools import combinations

# Assuming 'vocab_size' is 1024 and 'df' is your DataFrame with a column 'x' containing token lists
vocab_size = 1024

# Step 1: Initialize an empty co-occurrence matrix
co_occurrence_matrix = np.zeros((vocab_size, vocab_size), dtype=int)

# Step 2: Iterate over each sample and update the co-occurrence matrix
for tokens in samples['x']:
    # For each sample, get unique token combinations
    unique_pairs = combinations(set(tokens), 2)  # Set removes duplicates within a sample
    for i, j in unique_pairs:
        co_occurrence_matrix[i, j] += 1
        co_occurrence_matrix[j, i] += 1  # Ensure symmetry

# Step 3: Convert matrix to DataFrame
pair_freq_df = pd.DataFrame(
    [(i, j, co_occurrence_matrix[i, j]) for i in range(vocab_size) for j in range(i + 1, vocab_size)],
    columns=['Token1', 'Token2', 'Frequency']
)

# Display the results
import ace_tools as tools; tools.display_dataframe_to_user(name="Token Pair Frequency", dataframe=pair_freq_df)


ModuleNotFoundError: No module named 'ace_tools'